In [ ]:
!pip install valorant

In [ ]:
import os
import valorant

In [ ]:
os.environ["VALPY-KEY"] = "RGAPI-88d6f215-b22d-4c82-ba97-d73924fc6db4"

In [ ]:
KEY = os.environ["VALPY-KEY"]
client = valorant.Client(KEY, locale=None)

In [ ]:
client.get_user_by_name(name="viikset#BR1", route="americas")

In [ ]:
client.get_match(id="c229a683-a205-4543-a174-522956e8b72d")

In [ ]:
client.get_session()

In [ ]:
token = 'RGAPI-68e255f5-6d59-4152-b84b-35f67be5e3fe'

In [ ]:
puuid = 'puuid=t6hgE7vM78zeB7VZqIF9BV4VC3xqSFyBQ_QaAK_l2xRSfxTqaTmh0Adqwc7V7QamAPszSWlTyDf2lg'

In [ ]:
import requests
response = requests.get('https://br.api.riotgames.com/lol/summoner/v4/summoners/by-name/viikset', headers={'Authorization': f'Bearer token {token}'})

In [ ]:
response.json()

# Main

In [34]:
import os
import json
import time
import requests
import pandas as pd

from datetime import datetime

In [2]:
date_time = datetime.now()
                    
if time.tzname[0] == 'UTC':
    date_time = date_time - timedelta(hours=3)

In [89]:
datetime = date_time.strftime("%Y%m%dT%H:%M:%S")

In [73]:
timestamp = int(date_time.timestamp())

In [19]:
formated_datetime = date_time.strftime("%Y%m%d_%Hh%Mm%Ss")

In [4]:
users = {
    "viikset": "BR1",
    "TADALA": "TADAL",
    "DIGAOTJS": "BR1",
    "mBirth": "BR1",
    "Kubata": "4881"
}

In [5]:
puuids = {
    "viikset": "f2f7c867-27ba-5291-a23c-bbdcfc079e76",
    "TADALA": "35a2368e-586f-53ec-8ee8-ea9ebdca7d6f",
    "DIGAOTJS": "2c51a2e4-0dcf-53f5-ae85-cd4d7d79f663",
    "mBirth": "6c241ab0-1c2f-5d5b-8908-4e809df8f6ca",
    "Kubata": "b2310858-1741-5d6f-8499-cb1f85bd182a"
}

## Get Account Data

In [58]:
all_match_ids = []

df_account = pd.DataFrame()
df_mmr_data = pd.DataFrame()
df_mmr_history = pd.DataFrame()


for name, tag in users.items():
    
    # Accounts Data
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/account/{name}/{tag}")
    
    with open(f'account/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_account = pd.concat([df_account, pd.json_normalize(response.json()['data'])])
        
        
    # MMR Data
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr/na/{name}/{tag}")
    
    with open(f'mmr_data/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    df_mmr_data = pd.concat([df_mmr_data, pd.json_normalize(response.json()['data'])])  
        
    # MMR History
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v1/mmr-history/na/{name}/{tag}")
    
    with open(f'mmr_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
        
    temp_mmr_history = pd.json_normalize(response.json()['data'])
    temp_mmr_history['name'] = response.json()['name']
    temp_mmr_history['tag'] = response.json()['tag']
    
    df_mmr_history = pd.concat([df_mmr_history, temp_mmr_history])
    
    # Match History
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v3/matches/na/{name}/{tag}?filter=competitive")
    
    with open(f'match_history/{timestamp}_{name}_{tag}.json', 'w') as f:
        json.dump(response.json(), f)
        
    # Listing All Match IDs
    for i in response.json()['data']:
        all_match_ids.append(i['metadata']['matchid'])

# Get All Match History

In [69]:
all_match_ids = list(set(all_match_ids))

In [71]:
for matchid in all_match_ids:
    
    response = requests.get(f"https://api.henrikdev.xyz/valorant/v2/match/{matchid}")
    
    with open(f'match/{matchid}.json', 'w') as f:
        json.dump(response.json(), f)

In [140]:
response = requests.get(f"https://api.henrikdev.xyz/valorant/v3/matches/na/{name}/{tag}?filter=competitive")

In [143]:
pd.DataFrame(response.json()['data'])

,metadata,players,teams,rounds,kills
0,"{'map': 'Icebox', 'game_version': 'release-06....",{'all_players': [{'puuid': '55e10892-343a-52e1...,"{'red': {'has_won': True, 'rounds_won': 13, 'r...","[{'winning_team': 'Blue', 'end_type': 'Elimina...","[{'kill_time_in_round': 28002, 'kill_time_in_m..."
1,"{'map': 'Haven', 'game_version': 'release-06.0...",{'all_players': [{'puuid': '6c241ab0-1c2f-5d5b...,"{'red': {'has_won': False, 'rounds_won': 8, 'r...","[{'winning_team': 'Blue', 'end_type': 'Elimina...","[{'kill_time_in_round': 20520, 'kill_time_in_m..."
2,"{'map': 'Haven', 'game_version': 'release-06.0...",{'all_players': [{'puuid': 'f2f7c867-27ba-5291...,"{'red': {'has_won': True, 'rounds_won': 13, 'r...","[{'winning_team': 'Blue', 'end_type': 'Bomb de...","[{'kill_time_in_round': 38108, 'kill_time_in_m..."
3,"{'map': 'Lotus', 'game_version': 'release-06.0...",{'all_players': [{'puuid': '35a2368e-586f-53ec...,"{'red': {'has_won': True, 'rounds_won': 13, 'r...","[{'winning_team': 'Red', 'end_type': 'Eliminat...","[{'kill_time_in_round': 17162, 'kill_time_in_m..."
4,"{'map': 'Split', 'game_version': 'release-06.0...",{'all_players': [{'puuid': 'a21f61a0-56f3-5619...,"{'red': {'has_won': False, 'rounds_won': 5, 'r...","[{'winning_team': 'Blue', 'end_type': 'Bomb de...","[{'kill_time_in_round': 8835, 'kill_time_in_ma..."
